In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as plt

In [ ]:
df_df_1_org = pd.read_csv(r"..\..\data\raw\Digital_Footprints_pt1.txt")
df_df_2_org = pd.read_csv(r"..\..\data\raw\Digital_Footprints_pt2.txt")
df_cp_org = pd.read_csv(r"..\..\data\raw\Client_Profiles.txt")
df_er_org = pd.read_csv(r"..\..\data\raw\Experiment_Roster.txt")

In [ ]:
df_df = pd.concat([df_df_1_org, df_df_2_org], axis=0, ignore_index=True)
df_df = df_df.drop_duplicates()

df_df.head()
df_df_sorted_date = df_df.sort_values(by='date_time', ascending=True)
df_df.head(20)

def validating_steps(df):
    # Create an empty list to store valid rows
    valid_rows = []

    # Loop over each unique client_id
    for client_id in df["client_id"].unique():
        # Filter the data for the current client_id
        client_data = df[df["client_id"] == client_id]
        
        # Initialize step counts
        step_counts = {
            "confirm": 0,
            "step_3": 0,
            "step_2": 0,
            "step_1": 0,
            "start": 0
        }

        # Loop through each row in the client_data and increment step_count as necessary
        for _, row in client_data.iterrows():
            process_step = row["process_step"]
            
            if process_step in step_counts:
                step_counts[process_step] += 1

        # Now filter the client_data for each process step and get the last occurrence
        for step in ["confirm", "step_3", "step_2", "step_1", "start"]:
            step_data = client_data[client_data["process_step"] == step]
            
            # If there is data for this step, get the last entry
            if not step_data.empty:
                latest_row = step_data.loc[step_data["date_time"].idxmax()]  # Get the row with the latest date_time
                valid_rows.append(latest_row)

    # Create a new DataFrame from the valid rows
    df_valid = pd.DataFrame(valid_rows)
    
    return df_valid


In [ ]:
def validating_steps(df):
    # First, convert 'date_time' to datetime format if not already
    df['date_time'] = pd.to_datetime(df['date_time'])

    # List of process steps to consider
    process_steps = ['confirm', 'step_3', 'step_2', 'step_1', 'start']

    # We will first filter the DataFrame to only include the rows with the relevant process steps
    df_filtered = df[df['process_step'].isin(process_steps)]

    # Now, we group by 'client_id' and 'process_step', then find the latest 'date_time' for each group
    latest_rows = df_filtered.loc[df_filtered.groupby(['client_id', 'process_step'])['date_time'].idxmax()]
    latest_rows = latest_rows.sort_values(by="date_time", ascending = True)
    
    return latest_rows

validating_steps(df_df)


# filtering valid steps for client
def validating_steps(df):
    # Create an empty list to store the valid rows
    valid_rows = []

    # Define the process steps of interest
    process_steps = ["confirm", "step_3", "step_2", "step_1", "start"]

    # Loop over each unique client_id
    for client_id in df["client_id"].unique():
        # Filter the data for the current client_id
        client_data = df[df["client_id"] == client_id]
        
        # Loop over each process step
        for step in process_steps:
            # Filter the data for the current process step
            step_data = client_data[client_data["process_step"] == step]
            
            # If there is any data for this step, get the row with the latest date_time
            if not step_data.empty:
                latest_row = step_data.loc[step_data["date_time"].idxmax()]
                valid_rows.append(latest_row)

    # Create a DataFrame from the valid rows
    df_valid = pd.DataFrame(valid_rows)
    
    return df_valid

df_valid = validating_steps(df_df)

df_valid.shape


In [ ]:
df_cp = df_cp_org.copy()
df_cp.head()

In [ ]:
df_er = df_er_org.copy()
df_er = df_er.dropna()
df_er.head()